In [ ]:
# 📦 Installation des dépendances
!pip install -q ollama langchain-ollama langchain langchain-community beautifulsoup4 langgraph chromadb requests

# 🔄 Installation et démarrage d'Ollama sur Colab
import subprocess
import time
import os
import requests
from IPython.display import clear_output

print("🔧 Installation d'Ollama...")
!curl -fsSL https://ollama.com/install.sh | sh

# Démarrer Ollama en arrière-plan
print("🚀 Démarrage d'Ollama...")
process = subprocess.Popen(
    ["ollama", "serve"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    env={**os.environ, "OLLAMA_HOST": "0.0.0.0:11434"}
)
time.sleep(10)  # Plus de temps pour Colab

# Vérification du service
try:
    response = requests.get("http://localhost:11434/api/version", timeout=5)
    print("✅ Ollama démarré avec succès")
except:
    print("⚠️ Ollama pourrait encore démarrer...")
    time.sleep(5)

# ⬇️ Téléchargement du modèle
print("📥 Téléchargement du modèle...")
!ollama pull llama3.2

clear_output(wait=True)
print("✅ Installation terminée !")

✅ Installation terminée !


In [ ]:
# ==================== IMPORTS ET CONFIGURATION ====================
from typing import List, TypedDict, Optional
from langchain_core.documents import Document
from langchain_community.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import StateGraph, END
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import logging

# Configuration
MODEL_NAME = "llama3.2"
llm = ChatOllama(model=MODEL_NAME, temperature=0.3, base_url="http://localhost:11434")
embedding = OllamaEmbeddings(model=MODEL_NAME, base_url="http://localhost:11434")

# ==================== ÉTAT DE L'AGENT ====================
class AgentState(TypedDict):
    question: str
    documents: List[Document]
    rewritten_question: Optional[str]
    is_relevant: bool
    attempts: int
    max_attempts: int
    final_answer: str

# ==================== BASE DE CONNAISSANCES SIMPLE ====================
class SimpleKnowledgeBase:
    """Base de connaissances simple pour éviter les problèmes de scraping"""

    def __init__(self):
        self.documents = self._create_documents()
        self.vectorstore = None
        self._setup_vectorstore()

    def _create_documents(self) -> List[Document]:
        """Crée des documents de base sur l'IA"""
        knowledge = [
            {
                "content": """L'intelligence artificielle (IA) est une technologie qui permet aux machines de simuler l'intelligence humaine. Elle inclut l'apprentissage automatique, le traitement du langage naturel, la vision par ordinateur et la robotique. L'IA peut être faible (spécialisée dans une tâche) ou forte (intelligence générale). Les applications incluent les assistants virtuels, les voitures autonomes, la reconnaissance vocale et les systèmes de recommandation.""",
                "source": "base_ia",
                "title": "Introduction à l'IA"
            },
            {
                "content": """L'apprentissage automatique (machine learning) est une sous-discipline de l'IA qui permet aux systèmes d'apprendre automatiquement à partir de données sans être explicitement programmés. Il existe trois types principaux : supervisé (avec étiquettes), non-supervisé (sans étiquettes) et par renforcement (avec récompenses). Les algorithmes populaires incluent les réseaux de neurones, les arbres de décision, et les SVM.""",
                "source": "base_ml",
                "title": "Machine Learning"
            },
            {
                "content": """Les réseaux de neurones artificiels s'inspirent du fonctionnement du cerveau humain. Ils sont composés de couches de neurones interconnectés qui traitent l'information. L'apprentissage profond utilise des réseaux multicouches pour résoudre des problèmes complexes. Les architectures populaires incluent les réseaux convolutifs (CNN) pour les images et les réseaux récurrents (RNN) pour les séquences.""",
                "source": "base_nn",
                "title": "Réseaux de Neurones"
            },
            {
                "content": """L'IA est présente dans notre quotidien : assistants vocaux (Siri, Alexa), recommandations Netflix, GPS avec optimisation d'itinéraires, filtrage des emails, traduction automatique, reconnaissance faciale sur les photos, chatbots du service client, et algorithmes des réseaux sociaux. Ces applications facilitent nos tâches quotidiennes et améliorent notre expérience utilisateur.""",
                "source": "base_applications",
                "title": "IA au quotidien"
            }
        ]

        docs = []
        for item in knowledge:
            doc = Document(
                page_content=item["content"],
                metadata={"source": item["source"], "title": item["title"]}
            )
            docs.append(doc)
        return docs

    def _setup_vectorstore(self):
        """Configuration de la base vectorielle"""
        try:
            print("🔧 Configuration de la base vectorielle...")

            # Diviser en chunks plus petits
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=400,
                chunk_overlap=50,
                length_function=len
            )

            splits = text_splitter.split_documents(self.documents)
            print(f"📄 {len(splits)} chunks créés")

            # Créer la base vectorielle
            self.vectorstore = Chroma.from_documents(
                documents=splits,
                embedding=embedding,
                collection_name="simple_ia_kb"
            )
            print("✅ Base vectorielle prête")

        except Exception as e:
            print(f"❌ Erreur vectorielle: {e}")
            self.vectorstore = None

    def search(self, query: str, k: int = 2) -> List[Document]:
        """Recherche dans la base"""
        if not self.vectorstore or not query.strip():
            return []

        try:
            retriever = self.vectorstore.as_retriever(search_kwargs={"k": k})
            return retriever.invoke(query)
        except Exception as e:
            print(f"⚠️ Erreur de recherche: {e}")
            return []

# ==================== INITIALISATION ====================
print("🔧 Initialisation de la base de connaissances...")
knowledge_base = SimpleKnowledgeBase()

# ==================== FONCTIONS DE L'AGENT ====================
def retrieve_documents(state: AgentState) -> dict:
    """Récupère les documents pertinents"""
    question = state.get("rewritten_question") or state["question"]
    documents = knowledge_base.search(question)
    print(f"📚 {len(documents)} documents trouvés")
    return {"documents": documents}

def evaluate_relevance(state: AgentState) -> dict:
    """Évalue la pertinence des documents"""
    docs = state["documents"]

    if not docs:
        return {"is_relevant": False}

    # Simple heuristique : si on a des documents, ils sont probablement pertinents
    return {"is_relevant": len(docs) > 0}

def rewrite_question(state: AgentState) -> dict:
    """Reformule la question"""
    try:
        original = state["question"]

        prompt = ChatPromptTemplate.from_messages([
            ("system", "Reformule cette question sur l'intelligence artificielle pour améliorer la recherche. Garde le sens original mais rends-la plus précise."),
            ("human", f"Question: {original}")
        ])

        chain = prompt | llm | StrOutputParser()
        rewritten = chain.invoke({})

        return {
            "rewritten_question": rewritten.strip(),
            "attempts": state["attempts"] + 1
        }
    except Exception as e:
        print(f"⚠️ Erreur reformulation: {e}")
        return {
            "rewritten_question": state["question"],
            "attempts": state["attempts"] + 1
        }

def generate_answer(state: AgentState) -> dict:
    """Génère la réponse finale"""
    try:
        question = state.get("rewritten_question") or state["question"]
        docs = state["documents"]

        if not docs:
            return {"final_answer": "🤔 Je n'ai pas trouvé d'informations spécifiques sur ce sujet dans ma base de connaissances."}

        # Préparer le contexte
        context_parts = []
        for i, doc in enumerate(docs[:2], 1):  # Limiter à 2 documents
            content = doc.page_content[:300]  # Limiter la longueur
            context_parts.append(f"Source {i}: {content}")

        context = "\n\n".join(context_parts)

        prompt = ChatPromptTemplate.from_messages([
            ("system", """Tu es un expert en intelligence artificielle. Réponds de manière claire, concise et pédagogique en français.
            Utilise les informations fournies et structure ta réponse avec des exemples concrets quand c'est possible."""),
            ("human", f"Question: {question}\n\nInformations disponibles:\n{context}\n\nRéponds en français:")
        ])

        chain = prompt | llm | StrOutputParser()
        answer = chain.invoke({})

        return {"final_answer": f"🤖 {answer.strip()}"}

    except Exception as e:
        print(f"⚠️ Erreur génération: {e}")
        return {"final_answer": f"❌ Erreur lors de la génération de la réponse: {str(e)}"}

def should_continue(state: AgentState) -> str:
    """Décide de la suite du workflow"""
    if state["is_relevant"] or state["attempts"] >= state["max_attempts"]:
        return "generate"
    return "rewrite"

# ==================== CRÉATION DU WORKFLOW ====================
print("🔗 Configuration du workflow...")

workflow = StateGraph(AgentState)
workflow.add_node("retrieve", retrieve_documents)
workflow.add_node("evaluate", evaluate_relevance)
workflow.add_node("rewrite", rewrite_question)
workflow.add_node("generate", generate_answer)

workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "evaluate")
workflow.add_conditional_edges(
    "evaluate",
    should_continue,
    {"rewrite": "rewrite", "generate": "generate"}
)
workflow.add_edge("rewrite", "retrieve")
workflow.add_edge("generate", END)

agent = workflow.compile()
print("✅ Agent RAG prêt !")

# ==================== FONCTIONS DE TEST ====================
def tester_agent():
    """Test automatique simplifié"""
    print("\n🧪 **TEST DE L'AGENT**")
    print("=" * 50)

    questions = [
        "Qu'est-ce que l'intelligence artificielle ?",
        "Explique-moi le machine learning",
        "Comment fonctionnent les réseaux de neurones ?",
        "Donne des exemples d'IA au quotidien"
    ]

    for i, question in enumerate(questions, 1):
        print(f"\n--- Test {i}/{len(questions)} ---")
        print(f"❓ {question}")

        try:
            state = {
                "question": question,
                "documents": [],
                "rewritten_question": None,
                "is_relevant": False,
                "attempts": 0,
                "max_attempts": 2,
                "final_answer": ""
            }

            result = agent.invoke(state)
            print(f"\n{result['final_answer']}\n")

        except Exception as e:
            print(f"❌ Erreur: {e}")

def mode_interactif():
    """Mode conversation interactive"""
    print("\n💬 **MODE INTERACTIF**")
    print("Tapez 'quit' pour arrêter")
    print("=" * 40)

    while True:
        try:
            question = input("\n❓ Votre question: ").strip()

            if question.lower() in ['quit', 'exit', 'stop', 'q']:
                print("👋 Au revoir !")
                break

            if not question:
                continue

            print("🔄 Réflexion...")

            state = {
                "question": question,
                "documents": [],
                "rewritten_question": None,
                "is_relevant": False,
                "attempts": 0,
                "max_attempts": 2,
                "final_answer": ""
            }

            result = agent.invoke(state)
            print(f"\n{result['final_answer']}\n")

        except KeyboardInterrupt:
            print("\n👋 Au revoir !")
            break
        except Exception as e:
            print(f"❌ Erreur: {e}")


🔧 Initialisation de la base de connaissances...
🔧 Configuration de la base vectorielle...
📄 6 chunks créés
✅ Base vectorielle prête
🔗 Configuration du workflow...
✅ Agent RAG prêt !


In [ ]:
# ==================== LANCEMENT ====================
print("\n🎯 **CHOISISSEZ VOTRE MODE:**")
print("1. Test automatique: tester_agent()")
print("2. Mode interactif: mode_interactif()")
print("\n📝 **Exemple d'utilisation:**")
print("tester_agent()  # Lance les tests")
print("# ou")
print("mode_interactif()  # Lance le mode conversation")

# Lancer le test automatique par défaut
tester_agent()


🎯 **CHOISISSEZ VOTRE MODE:**
1. Test automatique: tester_agent()
2. Mode interactif: mode_interactif()

📝 **Exemple d'utilisation:**
tester_agent()  # Lance les tests
# ou
mode_interactif()  # Lance le mode conversation

🧪 **TEST DE L'AGENT**

--- Test 1/4 ---
❓ Qu'est-ce que l'intelligence artificielle ?
📚 2 documents trouvés

🤖 L'intelligence artificielle (IA) est une branche de l'informatique qui concerne le développement d'algorithmes et de systèmes capables d'effectuer des tâches qui, à l'origine, nécessitaient un intelligence humaine. L'IA utilise les techniques de l'apprentissage automatique, du traitement du langage naturel, de la reconnaissance visuelle et d'autres domaines pour résoudre des problèmes complexes.

En fonction des informations disponibles, il est clair que l'intelligence artificielle est présente dans notre quotidien sous différentes formes. Voici quelques exemples concrets :

*   **Assistants virtuels** : comme Siri, Alexa ou Google Assistant, qui utilisent la